In [92]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Camera Calibration
***
We must first start with images that have all camera related distortion accounted for before further processing.

In [ ]:
def calibrate_camera(calibration_dir, checker_grid_size=(9,6)):
    # read collection of checkerboard images and extract corners
    objpoints = []
    imgpoints = []
    
    (grid_x, grid_y) = checker_grid_size
    objp = np.zeros((grid_x * grid_y, 3), np.float32)
    objp[:,:2] = np.mgrid[0:grid_x, 0:grid_y].T.reshape(-1, 2)
    
    for cal_file in os.listdir(calibration_dir):
        path = os.path.join(calibration_dir, cal_file)
        img = plt.imread(path)
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, checker_grid_size, None)
        
        if ret:
            imgpoints.append(corners)
            objpoints.append(objp)
        else:
            # Some images don't find the full 9x6 set of corners because they are cut off.
            # print here for warning though it isn't catastrophic.
            print("Failed to find checkerboard corners!:", cal_file)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return (mtx, dist) if ret else None

def undistort_image(img, cam_matrix, dist_params):
    return cv2.undistort(img, cam_matrix, dist_params, None, cam_matrix)

(cam_matrix, dist_params) = calibrate_camera('camera_cal')


Failed to find checkerboard corners!: calibration1.jpg


### Example: (note the lines have straightened out)
***

In [ ]:
def before_after_plot(img_before, img_after, title_before, title_after, cmap_before=None, cmap_after=None):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img_before, cmap=cmap_before)
    ax1.set_title(title_before, fontsize=50)
    ax2.imshow(img_after, cmap=cmap_after)
    ax2.set_title(title_after, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)    

In [ ]:
def ex1():
    img = plt.imread('camera_cal/calibration5.jpg')
    before_after_plot(img, undistort_image(img, cam_matrix, dist_params), 'Original Image', 'Undistorted Image')
ex1()

# Perspective Transformation
***
It is easier to process the curvature of lane lines when viewed from the top down.  Let's use a perspective transform to accomplish this.  To do this, we need to establish a correspondence between four points in the camera image to where we want the four corresponding locations to be in the top down image.

In [ ]:
def ex2():
    img = plt.imread('test_images/straight_lines1.jpg')
    img = undistort_image(img, cam_matrix, dist_params)
    (height, width) = img.shape[:2]
    # bottom left, clockwise
    src_points = np.float32([(200, height-1), (590, 450), (690, 450), (1100, height-1)])
    dst_points = np.float32([(200, height-1), (200, 0),   (1000, 0),   (1000, height-1)])
    
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    # This will be useful later when we want to remap back to original image.
    Minv = cv2.getPerspectiveTransform(dst_points, src_points)
    
    for file in os.listdir('test_images'):
        img = plt.imread(os.path.join('test_images', file))
        top_down = cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
        cv2.polylines(img, [src_points.reshape((-1,1,2)).astype(np.int32)], True, color=(255,0,0), thickness=5)
        cv2.polylines(top_down, [dst_points.reshape((-1,1,2)).astype(np.int32)], True, color=(255,0,0), thickness=5)

        #before_after_plot(img, top_down, 'Undistorted', 'Warped top down')
    
    return (M, Minv)
    
(perspM, perspMinv) = ex2()

# Thresholding
***
Here, we do thresholding techniques to get a rough extraction of the lane lines.

In [135]:
def lane_line_extract(img, threshx=50, threshy=50, mag_thresh=(20, 255), dir_thresh=(0.7, 1.3), ksize=3):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s   = hls[:,:,2]
    
    sobelx = np.abs(cv2.Sobel(s, cv2.CV_64F, 1, 0, ksize=ksize))
    sobely = np.abs(cv2.Sobel(s, cv2.CV_64F, 0, 1, ksize=ksize))
    
    mag_sobel = np.sqrt(sobelx*sobelx + sobely*sobely)
    dir_sobel = np.arctan2(sobely, sobelx)
    
    scaled_sobel = np.uint8(255*mag_sobel/np.max(mag_sobel))
    
    binary = np.zeros_like(scaled_sobel)
    (mag_thresh_min, mag_thresh_max) = mag_thresh
    (dir_thresh_min, dir_thresh_max) = dir_thresh
    binary[(((scaled_sobel > mag_thresh_min) & (scaled_sobel <= mag_thresh_max)) &
            ((dir_sobel > dir_thresh_min) & (dir_sobel <= dir_thresh_max))) |
           ((sobelx > threshx) & (sobely > threshy))] = 1
    
    return binary

In [168]:
extracted_lanes_images = []
undistored_images      = []

for file in os.listdir('test_images'):
    img = plt.imread(os.path.join('test_images', file))
    img = undistort_image(img, cam_matrix, dist_params)
    undistored_images.append(img)
    ext = lane_line_extract(img)
    ext = cv2.warpPerspective(ext, perspM, img.shape[:2][::-1], flags=cv2.INTER_LINEAR)
    img = cv2.warpPerspective(img, perspM, img.shape[:2][::-1], flags=cv2.INTER_LINEAR)
    #before_after_plot(img, ext, 'Original', 'Extracted lanes', None, 'gray')
    #histogram = np.sum(ext[ext.shape[0]//2:,:], axis=0)
    #plt.figure()
    #plt.plot(histogram)
    extracted_lanes_images.append(ext)

# Lane Finding
***
Use a sliding window to find peaks in # of pixels that indicate likely line positions.  Follow those up the image.

In [169]:
def get_lane_line_pixels(img, nwindows=9, margin=100, minpix=50):
    """
    nwindows - # of sliding windows
    margin   - Set the width of the windows +/- margin
    minpix   - Set minimum number of pixels found to recenter window
    """
    histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero  = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds  = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return (leftx, lefty, rightx, righty, out_img)

In [170]:
def calc_poly_fit_and_curvature(ylen, width, leftx, lefty, rightx, righty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, ylen-1, ylen )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # calculate the radius of curvature
    ym_per_pix =  30 / 720 # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700 # meters per pixel in x dimension
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    (Al, Bl) = (left_fit_cr[0], left_fit_cr[1])
    (Ar, Br) = (right_fit_cr[0], right_fit_cr[1])
    ymax     = (ylen - 1) * ym_per_pix
    
    Rc_l = ((1 + (2*Al*ymax+Bl)**2)**1.5) / abs(2 * Al)
    Rc_r = ((1 + (2*Ar*ymax+Br)**2)**1.5) / abs(2 * Ar)
    
    img_center = width / 2
    lane_center_pix = (left_fitx[ylen-1] + right_fitx[ylen-1]) / 2
    dist_from_center = abs(img_center - lane_center_pix) * xm_per_pix 
    
    return (left_fitx, right_fitx, ploty, Rc_l, Rc_r, dist_from_center)

In [171]:
lane_lines = []

for ext_img in extracted_lanes_images:
    (leftx, lefty, rightx, righty, out_img) = get_lane_line_pixels(ext_img)
    (left_fitx, right_fitx, ploty, Rc_l, Rc_r, dist_from_center) = calc_poly_fit_and_curvature(
        ext_img.shape[0], ext_img.shape[1], leftx, lefty, rightx, righty)
    
    lane_lines.append((left_fitx, right_fitx, ploty, Rc_l, Rc_r, dist_from_center))
    
    continue

    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    plt.figure()
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

# Map lines back to road and color region

In [176]:
def remap_lines(orig, Minv, left_fitx, right_fitx, ploty):
    warp_zero = np.zeros_like(orig[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (orig.shape[1], orig.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(orig, 1, newwarp, 0.3, 0)
    
    return (result, color_warp)

In [177]:
def draw_text(img, text, org):
    cv2.putText(img, text, org, cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), thickness=2)

In [178]:
def display_stats(img, Rc_l, Rc_r, dist_from_center):
    draw_text(img, 'Radius of Curvature (left):  {:.2f} m.'.format(Rc_l), (20, 50))
    draw_text(img, 'Radius of Curvature (right): {:.2f} m.'.format(Rc_r), (20, 80))
    draw_text(img, 'Distance from center:        {:.2f} m.'.format(dist_from_center), (20, 110))

In [179]:
for (undist_img, (left_fitx, right_fitx, ploty, Rc_l, Rc_r, dist_from_center)) in zip(undistored_images, lane_lines):
    (result, color_warp) = remap_lines(undist_img, perspMinv, left_fitx, right_fitx, ploty)
    display_stats(result, Rc_l, Rc_r, dist_from_center)
    #before_after_plot(result, color_warp, 'mapping', 'top down')

# Video Processing Pipeline

In [180]:
# Import everythingneeded to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [181]:
g_Val = 0

In [182]:
def process_image(img):
    undist_img = undistort_image(img, cam_matrix, dist_params)
    ext = lane_line_extract(undist_img)
    ext = cv2.warpPerspective(ext, perspM, img.shape[:2][::-1], flags=cv2.INTER_LINEAR)
    (leftx, lefty, rightx, righty, _) = get_lane_line_pixels(ext)
    (left_fitx, right_fitx, ploty, Rc_l, Rc_r, dist_from_center) = calc_poly_fit_and_curvature(
        ext.shape[0], ext.shape[1], leftx, lefty, rightx, righty)
    (result, _) = remap_lines(undist_img, perspMinv, left_fitx, right_fitx, ploty)
    display_stats(result, Rc_l, Rc_r, dist_from_center)
    global g_Val
    g_Val += 1
    plt.imsave("raw_images/{}.jpg".format(g_Val), img)
    plt.imsave("processed_images/{}.jpg".format(g_Val), result)
    return result

In [183]:
def process_video(input, output):
    clip = VideoFileClip(input)
    clip_output = clip.fl_image(process_image) #NOTE: this function expects color images!!
    %time clip_output.write_videofile(output, audio=False)

In [70]:
process_video('project_video.mp4', 'output.mp4')

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [12:03<00:00,  1.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

Wall time: 12min 3s
